In [1]:
#importing requierd libraires
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

#keras imagedatagenerator as mentioned in the 6G7Z1003 High-Performance Computing and Big Data (1CWK50) HPC TensorFlow Task (50%) (Assessment Overview)
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")


Using TensorFlow backend.


In [4]:
model = tf.keras.models.Sequential()

#input layer with 96 by 96 pixel size and 3 bits of depth representing color with max pooling which prooved to be better than strides
#without either training takes to long because there are to many parameters, followed by batch normalization and droput as advised in the (Assessment Overview)
#upon reaserch I found that droupout of 20% gives good results on Caltech-101 so I used the same dropout rate, I also used 16 filters because
#(Assessment Overview) suggested depth over number of filters, activation used is relu because it is most widely used
model.add(tf.keras.layers.Conv2D(16, (5, 5),  input_shape=(96, 96, 3) ,activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.20))
    
#1rd hidden layer, 2nd convolutional layer, with 32 filters
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.20))
    
#2th hidden layer, 3rd convolutional layer with 64 filters
model.add(tf.keras.layers.Conv2D(64, (3, 3),  activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.20))


#3rdhidden layer, Dense with 2x number of output, also seemed to work well on Caltech-101 so I applied it here also
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())

#output, using softmax, also standard
model.add(tf.keras.layers.Dense(256, activation='softmax'))
#optimizer used is adam with categorical crossentropy, also used in Caltech-101 and found that it is most commonly recomended online, metrics measured as (Assessment Overview) requeired
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'crossentropy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 92, 92, 16)        1216      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 46, 46, 16)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 46, 46, 16)        64        
_________________________________________________________________
dropout_3 (Dropout)          (None, 46, 46, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 44, 44, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 22, 22, 32)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 22, 22, 32)       

In [5]:
#images, the whole dataset i.e. the map or folder is stored in the same folder as the notebook
#input size meaning the dimentions of the picture is defined by target size
#batch size determines the amount of pictures that imagedatagenerator will generate, not on the amount of inputs that will be fitted to the NN in one step
#validation split is 0.1 10% as (Assessment Overview) requiers, epoch were set at 100 and proved to be adequate
img_path = './256_ObjectCategories/'
target_size = (96,96)
batch_size = 128
epochs = 100
steps_per_epoch = 64
validation_steps = 20
validation_split = 0.1



#devides the whole training data into test and train sets, class mode is set to categorical because the loss is categorical crossentropy, if it were sparse categorical crossentropy the calss mode woud be spars categorical
#rescale, rotation range etc.. were most common recomendations for working with Caltech-101 and Caltech-256 which I found
train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split= validation_split,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        zoom_range=0.15,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split= validation_split)

train_generator = train_datagen.flow_from_directory(
        img_path,
        subset = 'training',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        img_path,
        subset = 'validation',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical')

#following 2 lines are used for tensorboard
log_dir = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

#main training function, with batch size of 2, meaning 2 inputs are fitted to the NN at one step,... using teansorboard for callbacks
H=model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        batch_size = 2,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        callbacks=[tensorboard_callback])





Found 27644 images belonging to 256 classes.
Found 2963 images belonging to 256 classes.
Epoch 1/100
64/64 [==============================] - 35s 543ms/step - loss: 5.3119 - accuracy: 0.0710 - crossentropy: 5.3119 - val_loss: 5.8139 - val_accuracy: 0.0293 - val_crossentropy: 5.8139
Epoch 2/100
64/64 [==============================] - 35s 539ms/step - loss: 4.8341 - accuracy: 0.1101 - crossentropy: 4.8341 - val_loss: 6.5526 - val_accuracy: 0.0269 - val_crossentropy: 6.5526
Epoch 3/100
64/64 [==============================] - 34s 531ms/step - loss: 4.6720 - accuracy: 0.1289 - crossentropy: 4.6720 - val_loss: 6.1904 - val_accuracy: 0.0253 - val_crossentropy: 6.1904
Epoch 4/100
64/64 [==============================] - 37s 585ms/step - loss: 4.4217 - accuracy: 0.1503 - crossentropy: 4.4217 - val_loss: 6.3498 - val_accuracy: 0.0322 - val_crossentropy: 6.3498
Epoch 5/100
64/64 [==============================] - 35s 542ms/step - loss: 4.3352 - accuracy: 0.1659 - crossentropy: 4.3352 - val_loss

64/64 [==============================] - 35s 549ms/step - loss: 2.3809 - accuracy: 0.4430 - crossentropy: 2.3809 - val_loss: 3.6087 - val_accuracy: 0.2999 - val_crossentropy: 3.6087
Epoch 84/100
64/64 [==============================] - 34s 539ms/step - loss: 2.4421 - accuracy: 0.4344 - crossentropy: 2.4421 - val_loss: 3.8054 - val_accuracy: 0.3019 - val_crossentropy: 3.8054
Epoch 85/100
64/64 [==============================] - 35s 544ms/step - loss: 2.3529 - accuracy: 0.4421 - crossentropy: 2.3529 - val_loss: 3.4401 - val_accuracy: 0.3354 - val_crossentropy: 3.4401
Epoch 86/100
64/64 [==============================] - 35s 546ms/step - loss: 2.3318 - accuracy: 0.4436 - crossentropy: 2.3318 - val_loss: 4.3912 - val_accuracy: 0.2256 - val_crossentropy: 4.3912
Epoch 87/100
64/64 [==============================] - 34s 539ms/step - loss: 2.3489 - accuracy: 0.4501 - crossentropy: 2.3489 - val_loss: 3.4013 - val_accuracy: 0.3431 - val_crossentropy: 3.4013
Epoch 88/100
64/64 [==================